In [19]:
import redis
import hiredis
import ranking
import os
import evaluate
from rq import Queue
import ast
from tournament import Team, Game
import pprint
import random
import statistics as stats
import math
pp = pprint.PrettyPrinter(indent=4)
pprint = pp.pprint
redis_host = 'polo-001.srx7vr.0001.apse1.cache.amazonaws.com'
redis_port = '6379'
wait_time = '600'
r = redis.StrictRedis(host=redis_host, port=redis_port, db=0)


def read_list(list_string):
    return map(int,list_string)


In [5]:
data_structure = {'s_ids':list,
                  't_ids':list,
                  'm_ids':list,
                  's':{'t_ids':list,
                       'game_ids':list,
                       'game':{ 'team_1':int,
                                'team_2':int,
                                'score_1':int,
                                'score_2':int,
                                'round':int,
                                's_updated':int,
                                'kl_updated':int,
                                'status':int}},
                  'team':{'name':str,'members':list},
                  'member':{'name':str,
                            'city':str}}

['score_1', 'score_2', 'team_1', 'team_2', 'updated', 'round']

In [6]:
session = {'s_ind':1}
s_ind = session['s_ind']

session_key = "".join(["s:",str(s_ind)])
game_properties = data_structure['s']['game'].keys()

class SessionConnection:

    def __init__(self,session_key,r_conn):
        self.session_key=session_key
        self.r_conn=r_conn 
    
    def team_list_key(self):
        return "".join([self.session_key,":t_ids:"])

    def game_key(self,game_id):
        return "".join([self.session_key,":game:",str(game_id)])
        
    def games_list_key(self):
        return "".join([self.session_key,":game_ids"])
    
    def strength_key(self,team_id):
        return "".join([self.session_key,":strengths:",str(team_id)])
    
    def kl_key(self,team1_id,team2_id):
        return "".join([self.session_key,":kl:t1:",str(team1_id),
                                           ':t2:',str(team2_id)])
    
    def property_key(self,game_id,prop):
        return "".join([self.game_key(game_id),':',prop])

    def get_team_list(self):
        return read_list(self.r_conn.lrange(self.team_list_key(),0,-1))

    def get_game_list(self):
        return read_list(self.r_conn.lrange(self.games_list_key(),0,-1))

    def get_game_property(self,id,prop):
        return int(self.r_conn.get(self.property_key(id,prop)))
    
    def get_game(self,game_id):
        game = {'id':game_id}
        for prop in game_properties:
            game[prop]=self.get_game_property(game_id,prop)
        return game
    
    def get_games(self):
        game_list = self.get_game_list()
        return dict(zip(game_list,
                        [self.get_game(id) for id in game_list]))
    
    def set_strengths(self,team_strengths):
        for team_id in team_stre:
            self.r_conn.set(self.strength_key(team_id),team_strengths[team_id])
      
    def set_kl(self,kl_graph):
        for edge in kl_graph:
            self.r_conn.set(self.kl_key(edge['t1'],edge['t2']),edge['weight'])
        
    def set_game_property(self,game_id,prop,val):
        self.r_conn.set(self.property_key(game_id,prop),val)

    def set_game(self,game):
        id = game['id']
        for prop in game:
            self.set_game_property(id,prop,game[prop])
    
    def make_teams(self,team_ids):

        def new_team(id,ind):
            t = Team()
            t.id = id
            t.session_id = ind
            return t
        
        return [new_team(ind,id) for ind,id in enumerate(team_ids)]
        
    def mcmc_games(self,games,teams):
        teams_dict = dict(zip([t.session_id for t in teams],teams))
        
        def new_game(id):
            g = Game()
            game = games[id]
            g.id = id
            g.teams = [teams_dict[game['team_1']],teams_dict[game['team_2']]]
            if not game['status'] == 1:
                g.scores = None
            else:
                g.scores = [game['score_1'],game['score_2']]
            g.round = game['round']
            return g
        
        return [new_game(id) for id in games]

In [9]:
dc = SessionConnection(session_key,r)
print(dc.games_list_key())
print(dc.team_list_key())
print(dc.game_key(1))
print(dc.strength_key(1))
print(dc.property_key(1,'team_1'))
dc.r_conn.lrange(dc.team_list_key(),0,-1)

s:1:game_ids
s:1:t_ids:
s:1:game:1
s:1:strengths:1
s:1:game:1:team_1


[]

In [10]:
dc.set_game_property(4,'score_1',-1)
dc.get_game_property(4,'score_1')

-1

In [21]:
r.delete(dc.team_list_key())
r.delete(dc.games_list_key())
test_games = {0:[1,0,1,3,1,0,0,1],
              1:[3,2,1,3,1,0,0,1],
              2:[2,0,1,3,2,0,0,1],
              3:[3,1,1,5,2,0,0,1],
              4:[2,1,0,0,3,0,0,0]}
def make_game(id,props):
    game = dict(zip(['team_1','team_2','score_1','score_2',
                     'round','s_updated','kl_updated','status'],props))
    game['id'] = id
    return game

for k in test_games:
   dc.set_game(make_game(k,test_games[k])) 
   r.rpush(dc.games_list_key(),k)

def set_team_ids(ids):
    for id in ids:
        r.rpush(dc.team_list_key(),id)
    
set_team_ids([0,1,2,3,4])
teams = dc.make_teams(dc.get_team_list())
games = dc.get_games()
sess = ranking.Seeding(teams)
sess.games = dc.mcmc_games(games,teams)
sess.games[4].scores
sess.mc_points=100000
sess.mc_burn = 10000
sess.fit_model()

running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 21.1 sec

In [26]:
def score_choices(max_score):
    choices = []
    for ii in range(0,max_score):
        for jj in range(0,max_score):
            choices = choices + [[ii,jj]]
    return choices

def discrete_lahiri(pr_f,choices):
    pr = 0
    u = 1
    while pr < u:
        x = random.choice(choices)
        pr = pr_f(x)
        u = random.uniform(0,1)
    return x

def generate_session(n_teams):
    teams = []
    strengths = [random.uniform(-2,2) for n in range(n_teams)]
    strengths = [s-stats.mean(strengths) for s in strengths]
    
    def m_team(id):
        t = Team()
        t.id = id
        t.session_id = id
        t.strength = strengths[id]
        return t
    teams = map(m_team, range(n_teams))    
    sess=ranking.Seeding(teams)
    return sess
        
def generate_round(sess,r):
    sess.update_KL_graph()
    games = sess.stage_round(r,sess.kl_info_dict())
    
    
    def prob_f(strengths,scores):
        return math.exp(sess.prob_func(strengths[1]-strengths[0],sess.scale,scores[1],scores[0]))
    
    def sample_scores(game):
        strengths = [t.strength for t in game.teams]
        pr_f = lambda x: prob_f(strengths,x)
        return discrete_lahiri(pr_f,score_choices(sess.max_score))
    
    for g in games:
        g.scores = sample_scores(g)
        
    sess.games = sess.games + games
    return sess


In [20]:
s = generate_session(18)
s.mc_points = 10000
s.mc_burn = 1000
s = generate_round(s,1)
s.fit_model()
s = generate_round(s,2)
s.fit_model()
s = generate_round(s,3)
s.fit_model()
s = generate_round(s,4)
s.fit_model()
s = generate_round(s,5)
s.fit_model()

running MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 2.3 secrunning MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 2.3 secrunning MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 2.4 secrunning MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 2.4 secrunning MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 2.5 sec

In [28]:
s = generate_session(28)
s.mc_points = 10000
s.mc_burn = 1000
s = generate_round(s,1)
s.fit_model()

%time games = s.update_KL_graph()


running MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 3.4 secCPU times: user 45.2 s, sys: 83.8 ms, total: 45.3 s
Wall time: 45.2 s


In [29]:
s = generate_session(28)
s.mc_points = 100000
s.mc_burn = 10000
s = generate_round(s,1)
s.fit_model()

%time games = s.update_KL_graph()


running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 34.4 secCPU times: user 7min 3s, sys: 182 ms, total: 7min 3s
Wall time: 7min 3s


In [49]:
    def kl_func(sess):
        scores = range(sess.max_score+1)
        f = sess.prob_func
        a = sess.mcmc.scale.trace[:]
        trace = sess.mcmc.marginal_delta.trace[:,:]
        exp = np.exp
        def kl(ind):
            def expect(fn):
                return np.mean(fn(trace[:,ind]))
            
            Epr = np.array([[expect( lambda x: exp(f(x,a,s1,s2)))
                 for s1 in scores] for s2 in scores])
            Elogpr = np.array([[expect( lambda x: f(x,a,s1,s2) )
                 for s1 in scores] for s2 in scores])
            Eprlogpr = np.array( [[expect(lambda x: exp(f(x,a,s1,s2))*f(x,a,s1,s2)) 
                 for s1 in scores ] for s2 in scores])
        
            return np.sum(Eprlogpr)-np.sum(Epr*Elogpr)
        return kl  
    
f = kl_func(s)
%time f(0)
%time s.KL_info_teams(0)

CPU times: user 1.13 s, sys: 41 µs, total: 1.13 s
Wall time: 1.12 s
CPU times: user 1.12 s, sys: 0 ns, total: 1.12 s
Wall time: 1.12 s


3.7783442869472035

In [60]:
import multiprocessing
def fun(f,q_in,q_out):
    while True:
        i,x = q_in.get()
        if i is None:
            break
        q_out.put((i,f(x)))

def parmap(f, X, nprocs = multiprocessing.cpu_count()):
    q_in   = multiprocessing.Queue(1)
    q_out  = multiprocessing.Queue()

    proc = [multiprocessing.Process(target=fun,args=(f,q_in,q_out)) for _ in range(nprocs)]
    for p in proc:
        p.daemon = True
        p.start()

    sent = [q_in.put((i,x)) for i,x in enumerate(X)]
    [q_in.put((None,None)) for _ in range(nprocs)]
    res = [q_out.get() for _ in range(len(sent))]

    [p.join() for p in proc]

    return [x for i,x in sorted(res)]



In [64]:
f = kl_func(s)
%time parmap(f,range(378))
%time s.fit_model()

CPU times: user 933 ms, sys: 121 ms, total: 1.05 s
Wall time: 5min 21s
running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 35.8 secCPU times: user 40.4 s, sys: 27.3 s, total: 1min 7s
Wall time: 35.8 s


In [963]:
pprint([[t.id, t.strength] for t in s.teams])
math.sqrt(sum([([t.strength for t in s.teams][ind]-st)**2 for ind,st in enumerate(s.strengths())]))/s.n_teams

[   [0, 1.4612338805761724],
    [1, -0.12282386997823952],
    [2, -1.070688662593081],
    [3, 1.5670693813850214],
    [4, -0.2144005184177908],
    [5, 0.0061031265798519585],
    [6, 1.2747228588012662],
    [7, -1.2827789506641838],
    [8, -0.5819467595480314],
    [9, 1.3481465571001405],
    [10, -0.8945452908674008],
    [11, -0.4808786111266578],
    [12, 1.5846592375584323],
    [13, -1.864252261068736],
    [14, 0.059112062224409956],
    [15, -1.7357795031306504],
    [16, 0.0992869726141477],
    [17, 0.847760350555328]]


0.23095517234868795

In [63]:
len(s.marginal_ind)

378

In [943]:
[[[t.id for t in g.teams],g.scores,g.round] for g in s.games]

[[[0, 17], [1, 2], 1],
 [[1, 16], [6, 0], 1],
 [[2, 15], [1, 5], 1],
 [[3, 14], [2, 5], 1],
 [[4, 13], [1, 3], 1],
 [[5, 12], [2, 5], 1],
 [[6, 11], [3, 5], 1],
 [[7, 10], [0, 2], 1],
 [[8, 9], [3, 4], 1],
 [[0, 14], [1, 3], 2],
 [[1, 17], [0, 3], 2],
 [[2, 9], [0, 9], 2],
 [[3, 7], [4, 0], 2],
 [[4, 16], [6, 0], 2],
 [[5, 6], [0, 5], 2],
 [[8, 15], [4, 1], 2],
 [[10, 13], [0, 8], 2],
 [[11, 12], [8, 0], 2],
 [[0, 13], [0, 7], 3],
 [[1, 12], [3, 3], 3],
 [[2, 10], [5, 0], 3],
 [[3, 11], [3, 6], 3],
 [[4, 17], [4, 1], 3],
 [[5, 7], [1, 3], 3],
 [[6, 16], [5, 0], 3],
 [[8, 14], [3, 3], 3],
 [[9, 15], [6, 1], 3],
 [[0, 1], [0, 3], 4],
 [[2, 14], [0, 2], 4],
 [[3, 5], [3, 0], 4],
 [[4, 12], [6, 0], 4],
 [[6, 13], [0, 5], 4],
 [[7, 17], [0, 5], 4],
 [[8, 16], [7, 1], 4],
 [[9, 11], [2, 1], 4],
 [[10, 15], [0, 7], 4],
 [[0, 10], [5, 0], 5],
 [[1, 11], [1, 4], 5],
 [[2, 17], [0, 5], 5],
 [[3, 8], [1, 6], 5],
 [[4, 9], [4, 0], 5],
 [[5, 16], [3, 7], 5],
 [[6, 7], [5, 0], 5],
 [[12, 15], [0, 4]

In [860]:
pr_f = lambda x: math.exp(s.prob_func(s.teams[3].strength-s.teams[1].strength,1.0,x[0],x[1]))

In [861]:
[discrete_lahiri(pr_f,score_choices(10)) for x in range(20)]

[[5, 2],
 [1, 0],
 [3, 2],
 [3, 4],
 [0, 6],
 [3, 5],
 [0, 6],
 [9, 6],
 [2, 5],
 [9, 9],
 [0, 0],
 [4, 0],
 [8, 6],
 [0, 6],
 [2, 3],
 [6, 6],
 [6, 0],
 [0, 3],
 [8, 9],
 [5, 5]]

In [859]:
s = generate_session(4)
s.scale

2.0

In [391]:
[t.id for t in sess.stage_round(3)[1].teams]

[2, 4]

In [892]:
x = random.choice(score_choices(10))
p = pr_f(x)
u = random.uniform(0,1)
print(x)
print(p)
print(u)

[0, 2]
0.0561291416822
0.787541383464


In [900]:
reload(ranking)
reload(worker)
reload(evaluate)
reload(conn)

<module 'conn' from 'conn.pyc'>

In [918]:
import rq
import worker
import evaluate
import time
import conn
import statistics as stats

In [950]:
queue = rq.Queue(connection=redis.Redis())
s.mc_points=100000
s.mc_burn = 10000
s.fit_model()


running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 80.0 sec

In [520]:
job = queue.enqueue_call(func=evaluate.update_results,args=(sess.games,sess))

In [519]:
job = queue.enqueue(evaluate.test_func)

In [642]:
evaluate.update_results(session_key,r)

running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 0.6 sec

AttributeError: 'int' object has no attribute 'info'

In [740]:
filter(lambda x: not x == 5,range(10))

[0, 1, 2, 3, 4, 6, 7, 8, 9]

In [468]:
print(job.result)

None


In [489]:
queue.get_job_ids()

[u'848e283d-0d6b-47e4-a7a1-9b37899dba0c']

In [524]:
import pickle

In [532]:
p = pickle.Pickler?

In [536]:
[g.id for g in sess.games]

[0, 1, 2, 3, 4]

In [549]:
g = sess.games[0]
g.

In [922]:
stats.mean([random.uniform(0,1) for x in range(10)])

0.6029027210959689

In [38]:
import numpy as np

In [39]:
np.mean

<function numpy.core.fromnumeric.mean>